In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, e0cd330e-2bfd-47f5-9422-a8714013f108, 3, Finished, Available, Finished)

###### **<mark>Creating Gold order_return Table</mark>**

In [2]:
%%sql
create table if not exists Ecommerce_project.Gold_OrderReturn
(
    Order_ID string,
    Return string,
    Order_year int,
    Order_Month int,
    Created_TS TIMESTAMP,
    Modified_TS TIMESTAMP
)
USING DELTA
PARTITIONED by (Order_year,Order_Month)

StatementMeta(, e0cd330e-2bfd-47f5-9422-a8714013f108, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

###### <mark>**Designing Incremental load for Gold_OrderReturn Table**</mark>

In [3]:
Max_Date=spark.sql("SELECT coalesce(max('Modified_TS'),'1900-01-01') from Ecommerce_project.Gold_orderReturn").first()[0]

StatementMeta(, e0cd330e-2bfd-47f5-9422-a8714013f108, 5, Finished, Available, Finished)

###### <mark>**Adding columns to data frame**</mark>

In [4]:
df=spark.sql(
"""SELECT Order_ID,
Return,
Order_year,
Order_Month,
Created_TS,
Modified_TS
FROM Ecommerce_project.bronze_sales where Return="Yes" and Modified_TS >'{}'""".format(Max_Date))

StatementMeta(, e0cd330e-2bfd-47f5-9422-a8714013f108, 6, Finished, Available, Finished)

###### **<mark>Storing this dataframe in temporary view</mark>**

In [5]:
df.createOrReplaceTempView("ViewReturn")

StatementMeta(, e0cd330e-2bfd-47f5-9422-a8714013f108, 7, Finished, Available, Finished)

###### **<mark>Inserting the data from view to Gold_OrderReturn Table</mark>**

In [6]:
%%sql
INSERT INTO Ecommerce_project.Gold_OrderReturn
SELECT * FROM ViewReturn

StatementMeta(, e0cd330e-2bfd-47f5-9422-a8714013f108, 8, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>